# Document retrieval from Wikipedia data

In [1]:
import turicreate

# Load some text data from Wikipedia

In [2]:
people = turicreate.SFrame('people_wiki.csv')

Finished parsing file /Users/zhanglibo/Desktop/python3/site-packages/people_wiki.csv

Parsing completed. Parsed 100 lines in 0.623348 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 26690 lines. Lines per second: 26416.3

Finished parsing file /Users/zhanglibo/Desktop/python3/site-packages/people_wiki.csv

Parsing completed. Parsed 59071 lines in 1.73402 secs.

In [3]:
people

URI,name,text
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...
<http://dbpedia.org/resource/Harpdog_Brown> ...,Harpdog Brown,harpdog brown is a singerand harmonica player who ...
<http://dbpedia.org/resource/Franz_Rottensteiner> ...,Franz Rottensteiner,franz rottensteiner bornin waidmannsfeld lower ...
<http://dbpedia.org/resource/G-Enka> ...,G-Enka,henry krvits born 30december 1974 in tallinn ...
<http://dbpedia.org/resource/Sam_Henderson> ...,Sam Henderson,sam henderson bornoctober 18 1969 is an ...
<http://dbpedia.org/resource/Aaron_LaCrate> ...,Aaron LaCrate,aaron lacrate is anamerican music producer ...
<http://dbpedia.org/resource/Trevor_Ferguson> ...,Trevor Ferguson,trevor ferguson aka johnfarrow born 11 november ...
<http://dbpedia.org/resource/Grant_Nelson> ...,Grant Nelson,grant nelson born 27april 1971 in london ...
<http://dbpedia.org/resource/Cathy_Caruth> ...,Cathy Caruth,cathy caruth born 1955 isfrank h t rhodes ...


# Explore data

In [4]:
people['word_count'] = turicreate.text_analytics.count_words(people['text'])
people['tfidf'] = turicreate.text_analytics.tf_idf(people['text'])

In [5]:
elton = people[people['name'] == 'Elton John']
elton_word_count_table = elton[['word_count']].stack('word_count', new_column_name = ['word', 'count'])
elton_tfidf_table = elton[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf'])

In [6]:
elton_word_count_table.sort('count', ascending = False)

word,count
the,27.0
in,18.0
and,15.0
of,13.0
a,10.0
has,9.0
john,7.0
he,7.0
on,6.0
award,5.0


In [7]:
elton_tfidf_table.sort('tfidf', ascending = False)

word,tfidf
furnish,18.38947183999428
elton,17.482320270031995
billboard,17.30368095754203
john,13.93931279239831
songwriters,11.250406447031539
tonightcandle,10.986495389225194
overallelton,10.986495389225194
19702000,10.293348208665249
fivedecade,10.293348208665249
aids,10.262846934045534


# Manually evaluate the distance between certain people's articles

In [8]:
victoria = people[people['name'] == 'Victoria Beckham']
paul = people[people['name'] == 'Paul McCartney']

elton_victoria_distance = turicreate.distances.cosine(elton['tfidf'][0], victoria['tfidf'][0])
elton_paul_distance = turicreate.distances.cosine(elton['tfidf'][0], paul['tfidf'][0])

print('Elton to Victoria Distance: ' + str(elton_victoria_distance))
print('Elton to Paul Distance: ' + str(elton_paul_distance))

Elton to Victoria Distance: 0.9567006376655429
Elton to Paul Distance: 0.8250310029221779


# Apply nearest neighbors for retrieval of Wikipedia articles

## Build the NN model

In [9]:
knn_word_count_model = turicreate.nearest_neighbors.create(people, label = 'name', 
                                                           features = ['word_count'], distance = 'cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [10]:
knn_tfidf_model = turicreate.nearest_neighbors.create(people, label = 'name', 
                                                      features = ['tfidf'], distance = 'cosine')

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

Done.

In [11]:
knn_word_count_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 27.541ms     |

| Done         |         | 100         | 434.123ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.220446049250313e-16,1
0,Cliff Richard,0.16142415258967036,2
0,Sandro Petrone,0.16822542751041114,3
0,Rod Stewart,0.16832716558706107,4
0,Malachi O'Doherty,0.177315545978884,5


In [12]:
knn_tfidf_model.query(elton)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.703ms     |

| Done         |         | 100         | 358.061ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.220446049250313e-16,1
0,Rod Stewart,0.7172196678927374,2
0,George Michael,0.7476009989692847,3
0,Sting (musician),0.7476719544306141,4
0,Phil Collins,0.7511932487904706,5


In [13]:
knn_word_count_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 16.793ms     |

| Done         |         | 100         | 271.851ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.220446049250313e-16,1
0,Mary Fitzgerald (artist),0.20730703611504997,2
0,Adrienne Corri,0.21450978278754795,3
0,Beverly Jane Fry,0.21746646874079278,4
0,Raman Mundair,0.21769547499150488,5


In [14]:
knn_tfidf_model.query(victoria)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 14.7ms       |

| Done         |         | 100         | 304.637ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.1102230246251565e-16,1
0,David Beckham,0.5481696102632145,2
0,Stephen Dow Beckham,0.7849867068283364,3
0,Mel B,0.8095855234085036,4
0,Caroline Rush,0.81982642291868,5
